# Heavy Hitters — 3アルゴリズム比較（同一データ）

入力: YouTubeライブチャット（列: `author`, `message`）。  
メトリクス: ground truth 上位20との重なり、順位相関、推定誤差。

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
from collections import Counter

# ファイルパスを正しいCSVファイルに修正し、区切り文字や引用符の処理を指定
# ここでは、実際のCSVファイル名を仮に "your_data.csv" としています。
# 実際のファイル名に合わせて修正してください。
try:
    df = pd.read_csv("/content/drive/MyDrive/大学/4年/ゼミ/watching_style_analysis/data/football/archive/Brighton vs. Manchester City _ LIVE Premier League Watch Along with @RantsNBantsClips_chat_log.csv",
                     encoding="utf-8",
                     engine="python",
                     sep=',',          # 区切り文字がカンマであることを明示
                     quotechar='"',    # 引用符がダブルクォーテーションであることを明示
                     on_bad_lines='skip' # エラーのある行をスキップ
                    )
    stream = df["author"].astype(str).tolist()
    N = len(stream)
    print("n =", N, "unique authors =", df['author'].nunique())

except FileNotFoundError:
    print("指定されたCSVファイルが見つかりません。ファイルパスを確認してください。")
except KeyError:
    print("データフレームに 'author' 列が見つかりません。CSVファイルの内容を確認してください。")
except Exception as e:
    print(f"データの読み込み中にエラーが発生しました: {e}")

n = 9645 unique authors = 595


データ読み込みのエラーが解消されました。次に、定義済みのアルゴリズムを使ってヘビーヒッターを検出し、評価を行います。

In [11]:

# Implementations (同じセルに定義)
from collections import defaultdict, Counter
import math

def misra_gries(stream, k):
    T = {}
    for x in stream:
        if x in T:
            T[x] += 1
        elif len(T) < k-1:
            T[x] = 1
        else:
            dead=[]
            for j in list(T.keys()):
                T[j]-=1
                if T[j]==0: dead.append(j)
            for j in dead:
                del T[j]
    return T

def lossy_counting(stream, k):
    T = {}
    Delta = 0
    n = 0
    for x in stream:
        n += 1
        if x in T: T[x]+=1
        else: T[x] = 1 + Delta
        if (n // k) != Delta:
            Delta = n // k
            for j in list(T.keys()):
                if T[j] < Delta:
                    del T[j]
    return T

def space_saving(stream, k):
    T = {}
    for x in stream:
        if x in T: T[x]+=1
        elif len(T)<k: T[x]=1
        else:
            j=min(T, key=T.get)
            m=T[j]
            del T[j]
            T[x]=m+1
    return T

def evaluate(T, truth, top=20):
    gt = [u for u,_ in truth.most_common(top)]
    cand = sorted(T.keys(), key=lambda u: truth[u], reverse=True)[:top]
    overlap = len(set(gt)&set(cand))
    return {
        "overlap@{}".format(top): overlap,
        "recall@{}".format(top): overlap/float(top),
        "size": len(T)
    }

from collections import Counter
truth = Counter(stream)
k=20

T_mg = misra_gries(stream, k)
T_lc = lossy_counting(stream, k)
T_ss = space_saving(stream, k)

print("MG  :", evaluate(T_mg, truth))
print("LC  :", evaluate(T_lc, truth))
print("SS  :", evaluate(T_ss, truth))

print("\nGround truth top-10:", truth.most_common(10))


MG  : {'overlap@20': 4, 'recall@20': 0.2, 'size': 16}
LC  : {'overlap@20': 7, 'recall@20': 0.35, 'size': 22}
SS  : {'overlap@20': 6, 'recall@20': 0.3, 'size': 20}

Ground truth top-10: [('@IbrahimOfficial1998', 279), ('@JESSEUPTOWN', 248), ('@Kn71-b2b', 242), ('@Woody1600', 236), ('@kloppsdownfall', 235), ('@Glo_Official1', 234), ('@rikkin4907', 221), ('@yashhgami', 178), ('@TheRapUpHaven', 159), ('@leotripoli', 156)]


コメント数が多い投稿者をトップ10で表示
